# ACESSIBILIDADE DA REDE DE VIÇOSA (COM ANEL)

### IMPORTANDO O PACOTE psycopg2 QUE CONECTA O PYTHON COM O POSTGRE-SQL E O PACOTE pandas PARA ORGANIZAR AS TABELAS DE ACESSIBILIDADE

In [1]:
import psycopg2 as pg
import pandas as pd

### CONECTANDO AO BANCO DE DADOS

In [2]:
con = pg.connect(host='localhost', 
                database='dissertacao_artigo1',
                user='postgres', 
                password='admin')

cur = con.cursor() #CRIANDO UMA INSTÂNCIA PARA EXECUTAR COMANDOS EM SQL

# OBS: O servidor hospedado na máquina local será conectado no banco de dados nomeado rede_exemplo, que possui usuário postgres e senha admin.

### VENDO QUANTOS GRAFOS A MALHA POSSUI

In [3]:
tabela_grafos = 'rede_vicosa_anel' #TABELA COM A REDE
sql = f'select max(id) from {tabela_grafos}' #COMANDO EM SQL A SER EXECUTADO
cur.execute(sql) #EXECUTANDO O COMANDO CRIADO
dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
id_max = e = dados_consultados[0][0] #ID MÁXIMO DA MALHA

### VENDO QUANTOS VÉRTICES A MALHA POSSUI 

In [4]:
tabela_vertices = 'rede_vicosa_anel_vertices_pgr' #TABELA COM A REDE
sql = f'select max(id) from {tabela_vertices}' #COMANDO EM SQL A SER EXECUTADO
cur.execute(sql) #EXECUTANDO O COMANDO CRIADO
dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
v = dados_consultados[0][0] #ID MÁXIMO DA MALHA

### CONECTIVIDADE DA MALHA

In [5]:
#INDICE ALFA: NÚMERO DE CLICOS DA REDE 'u'
#FORMULA:    u = e-v+1
#            u_max = 2v-5
#            alfa = u/u_max ou (e-v+1) / (2v-5)
#            sendo e: numero de linhas; v: numero de vértices

#INDICE BETA: SIMPLES RELAÇÃO ENTRE NUMERO DE LINHAS E VÉRTICES
#FORMULA:    beta = e / v

#INDICE GAMA: RELAÇÃO NUMERO DE LINHAS OBSERVADAS E O NUMERO MÁXIMO DE LINHAS
#FORMULA:    gama = e / (3(v-2))

alfa = (e-v+1)/(2*v-5)
beta = e/v
gama = e/(3*(v-2))

print(f'No cálculo de conectividade da rede, obtemos os seguintes resultados:\nÍndice Alfa: {alfa:.3f}\nÍndice Beta: {beta:.3f}\nÍndice Gama: {gama:.3f}')

No cálculo de conectividade da rede, obtemos os seguintes resultados:
Índice Alfa: 0.220
Índice Beta: 1.416
Índice Gama: 0.483


### NOME DAS TABELAS 

In [6]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos_acess = 'rede_vicosa_anel_mp' #NOME DE TABELA DA REDE
abr_grafos = 'rvmp' #ABREVIAÇÃO PARA A TABELA DA REDE
lista_custos = []

### LIGAÇÕES ENTRE OS GRAFOS

In [7]:
for id_i in range(1001, id_max + 1001): #ITERAÇÃO QUE IRÁ PERCORRER TODOS OS GRAFOS COMO VÉRTICE INICIAL

    lista_custos_i = [] #CRIANDO UMA LISTA VAZIA PARA RECEBER OS CUSTOS DO GRAFO i
    
    for id_j in range(1001, id_max + 1001): #ITERAÇÃO QUE IRÁ PERCORRER TODOS OS GRAFOS COMO VÉRTICE FINAL
        
        if id_i == id_j: #SE A ITERAÇÃO CALCULAR A DISTANCIA DE UM GRAFO PARA ELE MESMO, PULA A ITERAÇÃO E ADICIONA CUSTO ZERO NA LISTA
            lista_custos_i.append(0)
            continue
        
        else:
            
            sql = f"SELECT sum(djk.cost)/2 as tot_cost FROM pgr_dijkstra('SELECT id, source, target, cost, reverse_cost from {tabela_grafos_acess}', {id_i}, {id_j}, true) as djk JOIN {tabela_grafos_acess} {abr_grafos} ON djk.edge = {abr_grafos}.id;" #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O VÉRTICE INICIAL E FINAL DO GRAFO DO OBJETIVO FINAL.

            cur.execute(sql) #EXECUTANDO O COMANDO

            dados_consultados = cur.fetchall() #RETORNANDO OS DADOS            
                      
            cost = int(dados_consultados[0][0]) #CUSTO ENTRE i E j           
            
            #ADICIONANDO OS DADOS EM UMA LISTA:
            lista_custos_i.append(cost) #ADICIONANDO O CUSTO DE ATRAVESSAR DO GRAFO i ATÉ O GRAFO j EM UMA LISTA PROVISÓRIA

    lista_custos.append(lista_custos_i) #ADICIONANDO A TABELA PROVISIÓRIA ACIMA EM UMA LISTA QUE TERÁ TODAS INFORMAÇÕES


### TRANSFORMANDO A LISTA EM TABELA COM O PANDAS 

In [8]:
#CRIANDO UMA MATRIZ A PARTIR DA LISTA ACIMA:
matrizCustos = pd.DataFrame(lista_custos)

#SUBSTITUINDO O CABEÇALHO E LINHAS QUE ESTÁ INDO DE 0 ATÉ 7 PARA OS NOMES DOS GRAFOS QUE VAI DE A ATÉ H:
matrizCustos_ren = matrizCustos #CRIANDO UMA COPIA DA MATRIZ ANTIGA, PARA PRESERVAR A ESTRUTURA ORIGINAL

for i in range(0, len(matrizCustos)+1): #ITERAÇÃO PARA SUBSTITUIR O NOME DO CABEÇALHO E DAS LINHAS
    matrizCustos_ren = matrizCustos_ren.rename(columns={i: f'{i+1}'}, index = {i: f'{i+1}'})

In [9]:
matrizCustos

,0,1,2,3,4,5,6,7,8,9,...,116,117,118,119,120,121,122,123,124,125
0,0,3,14,2,3,10,1,3,3,2,...,5,6,8,1,6,4,2,3,5,7
1,3,0,16,5,1,13,4,2,1,2,...,8,9,9,4,7,7,4,3,7,9
2,8,11,0,9,11,1,8,11,11,10,...,12,10,15,7,9,11,10,11,13,14
3,2,5,15,0,5,11,1,5,5,4,...,4,7,9,2,8,3,4,5,6,6
4,3,1,15,5,0,13,4,3,2,2,...,8,9,8,4,6,7,3,2,7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,4,7,17,3,7,13,3,7,7,6,...,1,9,7,4,9,0,5,6,4,3
122,2,4,13,4,3,12,3,4,4,3,...,6,8,6,3,4,5,0,1,4,6
123,3,3,13,5,2,12,4,4,3,4,...,7,9,6,4,4,6,1,0,5,7
124,5,7,15,6,7,14,6,7,7,6,...,4,9,4,6,6,4,4,5,0,2


### Matriz dos custos de grafo a grafo 

In [10]:
matrizCustos_ren

,1,2,3,4,5,6,7,8,9,10,...,117,118,119,120,121,122,123,124,125,126
1,0,3,14,2,3,10,1,3,3,2,...,5,6,8,1,6,4,2,3,5,7
2,3,0,16,5,1,13,4,2,1,2,...,8,9,9,4,7,7,4,3,7,9
3,8,11,0,9,11,1,8,11,11,10,...,12,10,15,7,9,11,10,11,13,14
4,2,5,15,0,5,11,1,5,5,4,...,4,7,9,2,8,3,4,5,6,6
5,3,1,15,5,0,13,4,3,2,2,...,8,9,8,4,6,7,3,2,7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,4,7,17,3,7,13,3,7,7,6,...,1,9,7,4,9,0,5,6,4,3
123,2,4,13,4,3,12,3,4,4,3,...,6,8,6,3,4,5,0,1,4,6
124,3,3,13,5,2,12,4,4,3,4,...,7,9,6,4,4,6,1,0,5,7
125,5,7,15,6,7,14,6,7,7,6,...,4,9,4,6,6,4,4,5,0,2


### Acessibilidade pelo Método 1:

In [11]:
matrizAcess_conectiv = matrizCustos_ren.filter(items=list(map(lambda x: f'{x}', range(1, id_max + 1))))\
                                        .where(matrizCustos_ren.values == 1) #SELECIONANDO APENAS OS GRAFOS QUE POSSUEM LIGAÇÕES DIRETAS

matrizAcess_conectiv = matrizAcess_conectiv.apply(lambda x: x.replace(float('NaN'), 0)) #ATRIBUINDO VALOR ZERO PARA OS GRAFOS SEM LIGAÇÕES DIRETAS

In [12]:
matrizAcess_conectiv #MATRIZ DE CONECTIVADE

,1,2,3,4,5,6,7,8,9,10,...,117,118,119,120,121,122,123,124,125,126
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
Acess_1 = matrizAcess_conectiv.sum(axis=1) #SOMA DAS LINHAS DA MATRIZ DE CONECTIVIDADE
Acess_1 = pd.DataFrame(Acess_1).rename(columns = {0: 'SOMATÓRIO'}) #CRIANDO UM DATA FRAME PARA RECEBER OS DADOS

In [14]:
Acess_1

,SOMATÓRIO
1,4.0
2,4.0
3,2.0
4,3.0
5,4.0
...,...
122,4.0
123,4.0
124,4.0
125,4.0


### Acessibilidade pelo Método 2: 

In [15]:
Acess_2 = matrizCustos_ren.max() #VALORES MÁXIMOS PARA CADA GRAFO ATÉ O GRAFO MAIS DISTANTE NA REDE (NÚMERO ASSOCIADO)
Acess_2 = pd.DataFrame(Acess_2).rename(columns = {0: 'SOMATÓRIO'}) #NÚMERO ASSOCIADO

In [16]:
Acess_2

,SOMATÓRIO
1,11
2,14
3,20
4,12
5,14
...,...
122,14
123,13
124,14
125,16


### Acessibilidade pelo Método 3: 

In [17]:
# A ORDEM DE LIGAÇÃO MÁXIMA PODE SER OBTIDADE DA TABELA DO CÁLCULO DO NÚMERO ASSOCIADO, SENDO O VALOR MÁXIMO OBTIDO NO CALCULO DE ACESSIBILIDADE DO MÉTODO 2
lig_max = int(Acess_2.max())

lista_acess_3 = [] #CRIANDO UMA LISTA QUE RECEBERÁ O SOMATÓRIO DA ACESSIBILIDADE PARA CADA ORDEM

for i in range(1, lig_max + 1): #ITERAÇÃO QUE IRÁ PERCORRER DA ORDEM 1 ATÉ A ORDEM MÁXIMA
    
    matrizAcess_ordem_i = matrizCustos_ren.filter(items=list(map(lambda x: f'{x}', range(1, id_max + 1))))\
                                            .where(matrizCustos_ren.values == i) #SELECIONANDO APENAS OS GRAFOS QUE POSSUEM LIGAÇÕES DE ORDEM i
    
    matrizAcess_ordem_i = matrizAcess_ordem_i.apply(lambda x: x.replace(float('NaN'), 0)) #DEFININDO OS VALORES 'NaN' IGUAL A ZERO
    
    sumMatrizAcess_ordem_i = matrizAcess_ordem_i.sum(axis=1) #SOMANDO AS LINHAS
    
    lista_acess_3.append(list(sumMatrizAcess_ordem_i)) #ADICIONANDO O SOMATÓRIO A LISTA DE ACESSIBILIDADE 3

Acess_3 = pd.DataFrame(pd.DataFrame(lista_acess_3).sum()) #DEFININDO ACESSIBILIDADE PELO MÉTODO TRÊS COMO O SOMATÓRIO DE TODAS ACESSIBILIDADES DE ORDEM N

#RENOMEANDO AS LINHAS E COLUNAS DESSA MATRIZ:


Acess_3 = Acess_3.rename(columns = {0: 'SOMATÓRIO'}) #DEFININDO O NOME DA COLUNA

for i in range(0, len(Acess_3)): #ITERAÇÃO PARA SUBSTITUIR DAS LINHAS
    Acess_3 = Acess_3.rename(index = {i: f'{i+1}'})
    

## RESULTADOS: 

### Acessibilidade Método 1: 

In [18]:
Acess_1

,SOMATÓRIO
1,4.0
2,4.0
3,2.0
4,3.0
5,4.0
...,...
122,4.0
123,4.0
124,4.0
125,4.0


### Acessibilidade Método 2: 

In [19]:
Acess_2

,SOMATÓRIO
1,11
2,14
3,20
4,12
5,14
...,...
122,14
123,13
124,14
125,16


### Acessibilidade Método 3: 

In [20]:
Acess_3

,SOMATÓRIO
1,759.0
2,1022.0
3,1194.0
4,823.0
5,978.0
...,...
122,922.0
123,789.0
124,849.0
125,859.0


## Adicionando as Acessibilidades na tabela rede_vicosa: 

In [21]:
for id_i in range (1, id_max + 1):

    sql = f"update {tabela_grafos} set acess_1 = '{Acess_1.values[id_i-1][0]}', acess_2 = '{Acess_2.values[id_i-1][0]}', acess_3 = '{Acess_3.values[id_i-1][0]}' where id = {id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA 'REDE_VICOSA' AS ACESSIBILIDADES DAS VIAS.

    cur.execute(sql) #EXECUTANDO O COMANDO

    con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO
    
    print(f'Grafo id {id_i}: Acessibilidade 1: {Acess_1.values[id_i-1][0]}; Acessibilidade 2: {Acess_2.values[id_i-1][0]}; Acessibilidade 3: {Acess_3.values[id_i-1][0]};')
    
cur.close() #ENCERRANDO A INSTÂNCIA CRIADA PARA A EXECUÇÃO DO COMANDO

con.close() #ENCERRANDO A CONEXÃO COM BANCO DE DADOS

Grafo id 1: Acessibilidade 1: 4.0; Acessibilidade 2: 11; Acessibilidade 3: 759.0;
Grafo id 2: Acessibilidade 1: 4.0; Acessibilidade 2: 14; Acessibilidade 3: 1022.0;
Grafo id 3: Acessibilidade 1: 2.0; Acessibilidade 2: 20; Acessibilidade 3: 1194.0;
Grafo id 4: Acessibilidade 1: 3.0; Acessibilidade 2: 12; Acessibilidade 3: 823.0;
Grafo id 5: Acessibilidade 1: 4.0; Acessibilidade 2: 14; Acessibilidade 3: 978.0;
Grafo id 6: Acessibilidade 1: 2.0; Acessibilidade 2: 16; Acessibilidade 3: 1269.0;
Grafo id 7: Acessibilidade 1: 4.0; Acessibilidade 2: 11; Acessibilidade 3: 765.0;
Grafo id 8: Acessibilidade 1: 2.0; Acessibilidade 2: 14; Acessibilidade 3: 1065.0;
Grafo id 9: Acessibilidade 1: 4.0; Acessibilidade 2: 14; Acessibilidade 3: 1022.0;
Grafo id 10: Acessibilidade 1: 4.0; Acessibilidade 2: 13; Acessibilidade 3: 947.0;
Grafo id 11: Acessibilidade 1: 3.0; Acessibilidade 2: 15; Acessibilidade 3: 1088.0;
Grafo id 12: Acessibilidade 1: 3.0; Acessibilidade 2: 11; Acessibilidade 3: 774.0;
Grafo i

### ENCERRANDO CONEXÃO 

In [25]:
cur.close()
con.close()